In [1]:
cd scripts/behavioural_cloning

[WinError 3] System nie może odnaleźć określonej ścieżki: 'scripts/behavioural_cloning'
c:\Users\czyjt\Projects\engineering-thesis\automobile-racing-toolbox\scripts\behavioural_cloning


In [3]:
from preprocessing import get_episode, process_episode
import pandas as pd
import numpy as np
from PIL import Image
from pathlib import Path
import os
import matplotlib.pyplot as plt 
import cv2

# REad preprocessed data

In [4]:
import tables 

In [7]:
hdf5_path = "C:\\Users\czyjt\Projects\engineering-thesis\\automobile-racing-toolbox\compressed_dataset.hdf5"
extendable_hdf5_file = tables.open_file(hdf5_path, mode='r')
images_hdf5 = extendable_hdf5_file.root.images[:]
actions_hdf5 = extendable_hdf5_file.root.actions[:]

In [37]:
extendable_hdf5_file.close() 

In [36]:
import torch as th
from torch import nn
import torch.nn.functional as F
device = th.device("cuda" if th.cuda.is_available() else "cpu")

In [ ]:
class MultiLabelnNN(nn.Module):
    def __init__(self, n_channels, n_outputs, w, h):
        self.feature_extractor =  nn.Sequential(
            nn.Conv2d(n_channels, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten()
        )

        def conv2d_size_out(size, kernel_size, stride):
            return (size - (kernel_size - 1) - 1) // stride  + 1

        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w, 3, 1)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h, 3, 1)))
        linear_input_size = convw * convh * 64

        self.classifier = nn.Sequential(
            nn.Linear(linear_input_size, n_outputs),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.to(device)
        x = self.feature_extractor(x)
        return self.classifier(x)
